In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# 데이터프레임 로드
df = pd.read_csv('naver_category_5 동사 2개 이거나 형용사 2개.csv')

df

,review,category,restaurant
0,마제소바 준비하신다고 테스트로 맛보여주셔서,맛,윤스쿡
1,양도 많고 맛도 있어요,맛,윤스쿡
2,양많고 맛나요,맛,윤스쿡
3,학교앞 돈가스중 제일 맛있움ㅜ,맛,윤스쿡
4,너무 맛나네요,맛,윤스쿡
...,...,...,...
672,예전보단 많이가격이 올랐,가격,연어시장 석계본점
673,물가상승으로 전보다 가격은 올랐,가격,연어시장 석계본점
674,전에 23000원할따 갔었는데 가격 좀 내려서,가격,연어시장 석계본점
675,15900원에 연어무한이라 연어좋아하는 신랑이랑 자주 가는곳더이상은 가격이 오르지,가격,연어시장 석계본점


In [2]:
# 데이터프레임 로드
df1 = pd.read_csv('kakao_final.csv')

df1

,review,category,restaurant
0,맛 전혀 특별하지 않음,맛,무명칼국수
1,해물칼국수 정말 괜찮음 닭칼은 그닥 만두는 부추맛이 강해서,맛,무명칼국수
2,Strong point 1 쫄깃한 식감의 수타면 2 아삭하고 감칠맛 나는 겉절이 반...,맛,무명칼국수
3,찐맛집 닭칼이랑 해물칼국수 둘다,맛,무명칼국수
4,면이랑 국물 미쳤어요 김치도 대존맛 만두도 진짜임 광운대 가면 무조건 가야되는 맛집...,맛,무명칼국수
...,...,...,...
170,뭔가 저 말투랑 매너가 기분이 안좋았음 사장과 별개로 파스타는 진짜 개존맛이라 별3개,서비스,라체나1907
171,직원분이 친절하시고 음식의 양도 적당하고,서비스,라체나1907
172,서비스가 별로에요,서비스,라체나1907
173,서비스가 좋아요,서비스,라체나1907


In [3]:
# 데이터프레임 로드
import pandas as pd

# 데이터프레임 로드
df2 = pd.read_csv('labeled_review_data.csv')

# label 별 갯수 확인
label_counts = df2['label'].value_counts()

print(label_counts)

#부정 1173 , 긍정 721

label
0    1173
1     721
Name: count, dtype: int64


In [4]:
!pip install accelerate
!pip install transformers
!pip install transformers[torch]

In [5]:
!pip install transformers==4.12.0 accelerate==0.21.0

  Using cached transformers-4.12.0-py3-none-any.whl (3.1 MB)
  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [6]:
!pip install cmake cython
!pip install tokenizers==0.10.3

  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [7]:
!pip install --upgrade setuptools
!pip install tokenizers==0.10.3

  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [8]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# labeled 데이터를 로드합니다.
df2 = pd.read_csv('labeled_review_data.csv')

# 데이터를 학습 및 테스트 세트로 나눕니다.
train_df, test_df = train_test_split(df2, test_size=0.2, random_state=42)

model_names = [
    "monologg/kobert",
    "monologg/koelectra-base-v3-discriminator",
    "monologg/distilkobert",
    "monologg/koelectra-small-v3-discriminator",
    "bert-base-multilingual-cased"  # Multilingual BERT 추가
]

tokenizer_names = {
    "monologg/kobert": "monologg/kobert",
    "monologg/koelectra-base-v3-discriminator": "monologg/koelectra-base-v3-discriminator",
    "monologg/distilkobert": "monologg/distilkobert",
    "monologg/koelectra-small-v3-discriminator": "monologg/koelectra-small-v3-discriminator",
    "bert-base-multilingual-cased": "bert-base-multilingual-cased"  # Multilingual BERT 토크나이저
}


# 모델마다 학습 및 평가 진행
for model_name in model_names:
    print(f"Training and evaluating {model_name}...")

    # 토크나이저와 모델 로드
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # 데이터셋 생성
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # 전처리 함수
    def preprocess_function(examples):
        return tokenizer(examples['reviews'], padding='max_length', truncation=True)

    # 데이터셋 전처리
    encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
    encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)

    # Trainer 설정을 딕셔너리로 전달
    training_args = {
        "output_dir": f'./results_{model_name}',
        "evaluation_strategy": "epoch",
        "per_device_train_batch_size": 16,
        "per_device_eval_batch_size": 16,
        "num_train_epochs": 3,
        "weight_decay": 0.01,
        "logging_dir": f'./logs_{model_name}',
    }

    # Trainer 객체 초기화
    trainer = Trainer(
        model=model,
        args=TrainingArguments(**training_args),  # 딕셔너리를 TrainingArguments 객체로 변환하여 전달
        train_dataset=encoded_train_dataset,
        eval_dataset=encoded_test_dataset,
        compute_metrics=lambda pred: {"accuracy": accuracy_score(pred.label_ids, pred.predictions.argmax(axis=1))},
    )

    # 모델 학습
    trainer.train()

    # 모델 평가
    eval_result = trainer.evaluate()
    print(f"Evaluation results for {model_name}: {eval_result}")

    # 테스트 데이터에 대해 감성 분석 수행
    def predict_sentiment(review):
        inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(trainer.args.device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        return predictions.item()

    # 테스트 데이터 예측 결과 저장
    test_df[f'{model_name}_predictions'] = test_df['reviews'].apply(predict_sentiment)

    # 정확도 출력
    accuracy = accuracy_score(test_df['label'], test_df[f'{model_name}_predictions'])
    print(f"Accuracy for {model_name}: {accuracy}")

    # 정밀도, 재현율, F1-score 출력
    precision, recall, f1, _ = precision_recall_fscore_support(test_df['label'], test_df[f'{model_name}_predictions'], average='binary')
    print(f"Precision for {model_name}: {precision}, Recall: {recall}, F1-score: {f1}")

# 결과 출력
test_df.head()

Training and evaluating monologg/kobert...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:  63%|######2   | 231M/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1515 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.528831,0.746702
2,No log,0.547051,0.770449
3,No log,0.568025,0.783641


Evaluation results for bert-base-multilingual-cased: {'eval_loss': 0.5680245161056519, 'eval_accuracy': 0.783641160949868, 'eval_runtime': 11.3867, 'eval_samples_per_second': 33.285, 'eval_steps_per_second': 2.108, 'epoch': 3.0}
Accuracy for bert-base-multilingual-cased: 0.783641160949868
Precision for bert-base-multilingual-cased: 0.7394366197183099, Recall: 0.7, F1-score: 0.7191780821917808


,name,rating,reviews,label,monologg/kobert_predictions,monologg/koelectra-base-v3-discriminator_predictions,monologg/distilkobert_predictions,monologg/koelectra-small-v3-discriminator_predictions,bert-base-multilingual-cased_predictions
1054,패션5 한남점,3.0,너무 예쁘고 맛있어요 다만 가격이,0,0,1,0,1,1
611,봉산집,3.0,차돌된장밥소주,0,0,1,1,1,0
963,패션5 한남점,1.0,안가야징,0,0,0,1,0,0
1315,교다이야,5.0,정식 구색이 좋습니다 국물도 맛있도 면발도 이정도면 저는 충분하네요,1,0,1,0,1,1
585,봉산집,5.0,노포 차돌집 십년만에 온듯 여전히 맛있다,1,0,1,0,1,1


In [16]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# labeled 데이터를 로드합니다.
df2 = pd.read_csv('labeled_review_data.csv')

# 데이터를 학습 및 테스트 세트로 나눕니다.
train_df, test_df = train_test_split(df2, test_size=0.2, random_state=42)

# 모델 이름 및 토크나이저
model_name = "bert-base-multilingual-cased"
tokenizer_name = "bert-base-multilingual-cased"

# 토크나이저와 모델 로드
tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 클래스 레이블
class_labels = [0, 1]

# 클래스별 데이터 수 계산
class_counts = train_df['label'].value_counts(normalize=True)

# 클래스별 가중치 계산
class_weights_dict = {class_label: 1.0 / class_counts[class_label] for class_label in class_labels}

# 데이터셋 생성
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 전처리 함수
def preprocess_function(examples):
    return tokenizer(examples['reviews'], padding='max_length', truncation=True)

# 데이터셋 전처리
encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)

# 가중 평가 지표 계산 함수
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Trainer 설정을 딕셔너리로 전달
training_args = {
    "output_dir": f'./results_{model_name}',
    "evaluation_strategy": "epoch",
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 3,
    "weight_decay": 0.01,
    "logging_dir": f'./logs_{model_name}',
}

# Trainer 객체 초기화
trainer = Trainer(
    model=model,
    args=TrainingArguments(**training_args),
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    compute_metrics=compute_metrics,  # 가중 평가 지표 계산 함수 사용
)

# 모델 학습
trainer.train()

# 모델 평가
eval_result = trainer.evaluate()
print(f"Evaluation results for {model_name}: {eval_result}")

# 테스트 데이터에 대해 감성 분석 수행
def predict_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(trainer.args.device) for k, v in inputs.items()}
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    return predictions.item()

# 테스트 데이터 예측 결과 저장
test_df[f'{model_name}_predictions'] = test_df['reviews'].apply(predict_sentiment)

# 정확도 출력
accuracy = accuracy_score(test_df['label'], test_df[f'{model_name}_predictions'])
print(f"Accuracy for {model_name}: {accuracy}")

# 정밀도, 재현율, F1-score 출력
precision, recall, f1, _ = precision_recall_fscore_support(test_df['label'], test_df[f'{model_name}_predictions'], average='binary')
print(f"Precision for {model_name}: {precision}, Recall: {recall}, F1-score: {f1}")

# 결과 출력
test_df.head()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1515 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.598441,0.686016,0.674157,0.400000,0.502092
2,No log,0.498595,0.754617,0.752212,0.566667,0.646388
3,No log,0.485629,0.788918,0.773438,0.660000,0.712230


Evaluation results for bert-base-multilingual-cased: {'eval_loss': 0.4856291711330414, 'eval_accuracy': 0.7889182058047494, 'eval_precision': 0.7734375, 'eval_recall': 0.66, 'eval_f1': 0.7122302158273381, 'eval_runtime': 11.4193, 'eval_samples_per_second': 33.189, 'eval_steps_per_second': 2.102, 'epoch': 3.0}
Accuracy for bert-base-multilingual-cased: 0.7889182058047494
Precision for bert-base-multilingual-cased: 0.7734375, Recall: 0.66, F1-score: 0.7122302158273381


,name,rating,reviews,label,bert-base-multilingual-cased_predictions
1054,패션5 한남점,3.0,너무 예쁘고 맛있어요 다만 가격이,0,1
611,봉산집,3.0,차돌된장밥소주,0,0
963,패션5 한남점,1.0,안가야징,0,0
1315,교다이야,5.0,정식 구색이 좋습니다 국물도 맛있도 면발도 이정도면 저는 충분하네요,1,1
585,봉산집,5.0,노포 차돌집 십년만에 온듯 여전히 맛있다,1,1


In [20]:
#kakao final

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 모델 이름 및 토크나이저
model_name = "bert-base-multilingual-cased"
tokenizer_name = "bert-base-multilingual-cased"

# 토크나이저와 모델 로드
tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 감성 분석 함수 정의
def predict_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    return predictions.item()

# 리뷰에 대한 감성 분석 수행 및 레이블링
def label_sentiment(review):
    prediction = predict_sentiment(review)
    # 예측 결과에 따라 긍정(1) 또는 부정(0) 레이블 부여
    if prediction == 1:
        return 1  # 긍정
    else:
        return 0  # 부정

# 각 리뷰에 대해 레이블링을 수행하여 새로운 열에 추가
df1['label'] = df1['review'].apply(label_sentiment)

# 결과 출력
print(df1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                                                review category restaurant  \
0                                         맛 전혀 특별하지 않음        맛      무명칼국수   
1                     해물칼국수 정말 괜찮음 닭칼은 그닥 만두는 부추맛이 강해서        맛      무명칼국수   
2    Strong point 1 쫄깃한 식감의 수타면 2 아삭하고 감칠맛 나는 겉절이 반...        맛      무명칼국수   
3                                    찐맛집 닭칼이랑 해물칼국수 둘다        맛      무명칼국수   
4    면이랑 국물 미쳤어요 김치도 대존맛 만두도 진짜임 광운대 가면 무조건 가야되는 맛집...        맛      무명칼국수   
..                                                 ...      ...        ...   
170    뭔가 저 말투랑 매너가 기분이 안좋았음 사장과 별개로 파스타는 진짜 개존맛이라 별3개      서비스    라체나1907   
171                             직원분이 친절하시고 음식의 양도 적당하고      서비스    라체나1907   
172                                          서비스가 별로에요      서비스    라체나1907   
173                                           서비스가 좋아요      서비스    라체나1907   
174                                   사장님 너무 불친절해서 싫어요      서비스    라체나1907   

     label  
0        1  
1        1  
2        1  
3        1 

In [27]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# labeled 데이터를 로드합니다.
df10 = pd.read_csv('naver_final.csv')

# 모델 이름 및 토크나이저
model_name = "bert-base-multilingual-cased"
tokenizer_name = "bert-base-multilingual-cased"

# 토크나이저와 모델 로드
tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 감성 분석 함수 정의
def predict_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)  # softmax 적용
    positive_probability = probabilities[:, 1].item()  # 긍정 클래스의 확률
    return positive_probability

# 리뷰에 대한 감성 분석 수행 및 레이블링
def label_sentiment(review):
    positive_probability = predict_sentiment(review)
    # 긍정 확률이 0.5 이상이면 긍정(1), 아니면 부정(0)으로 레이블 부여
    if positive_probability >= 0.5:
        return 1  # 긍정
    else:
        return 0  # 부정

# 각 리뷰에 대해 레이블링을 수행하여 새로운 열에 추가
df10['label'] = df10['review'].apply(label_sentiment)

# 결과 출력
print(df10.head())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                    review category restaurant  label
0  마제소바 준비하신다고 테스트로 맛보여주셔서        맛        윤스쿡      1
1             양도 많고 맛도 있어요        맛        윤스쿡      1
2                  양많고 맛나요        맛        윤스쿡      1
3         학교앞 돈가스중 제일 맛있움ㅜ        맛        윤스쿡      1
4                  너무 맛나네요        맛        윤스쿡      1


In [25]:
# df1의 라벨에서 0과 1의 갯수 파악
label_counts = df1['label'].value_counts()
print(label_counts)


label
1    175
Name: count, dtype: int64


In [28]:
# df10의 라벨에서 0과 1의 갯수 파악
label_counts = df10['label'].value_counts()
print(label_counts)


label
1    697
Name: count, dtype: int64


In [32]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# labeled 데이터를 로드합니다.
df2 = pd.read_csv('labeled_review_data.csv')

# 데이터를 학습 및 테스트 세트로 나눕니다.
train_df, test_df = train_test_split(df2, test_size=0.2, random_state=42)

# 모델 이름 및 토크나이저
model_name = "monologg/koelectra-base-v3-discriminator"

# 토크나이저와 모델 로드
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 클래스 레이블
class_labels = [0, 1]

# 데이터셋 생성
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 전처리 함수
def preprocess_function(examples):
    return tokenizer(examples['reviews'], padding='max_length', truncation=True)

# 데이터셋 전처리
encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Trainer 설정을 딕셔너리로 전달
training_args = {
    "output_dir": "./results_koelectra",
    "evaluation_strategy": "epoch",
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 3,
    "weight_decay": 0.01,
    "logging_dir": "./logs_koelectra",
}

# Trainer 객체 초기화
trainer = Trainer(
    model=model,
    args=TrainingArguments(**training_args),
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    compute_metrics=compute_metrics,
)

# 모델 학습
trainer.train()

# 모델 평가
eval_result = trainer.evaluate()
print(f"Evaluation results for {model_name}: {eval_result}")

# 테스트 데이터에 대해 감성 분석 수행
def predict_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    return predictions.item()

# 테스트 데이터 예측 결과 저장
test_df['koelectra_predictions'] = test_df['reviews'].apply(predict_sentiment)

# 정확도 출력
accuracy = accuracy_score(test_df['label'], test_df['koelectra_predictions'])
print(f"Accuracy for KoELECTRA: {accuracy}")

# 정밀도, 재현율, F1-score 출력
precision, recall, f1, _ = precision_recall_fscore_support(test_df['label'], test_df['koelectra_predictions'], average='binary')
print(f"Precision for KoELECTRA: {precision}, Recall: {recall}, F1-score: {f1}")

# 결과 출력
print(test_df.head())


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1515 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.340582,0.865435,0.823529,0.840000,0.831683
2,No log,0.303892,0.873351,0.849315,0.826667,0.837838
3,No log,0.351410,0.886544,0.840764,0.880000,0.859935


Evaluation results for monologg/koelectra-base-v3-discriminator: {'eval_loss': 0.35141006112098694, 'eval_accuracy': 0.8865435356200527, 'eval_precision': 0.8407643312101911, 'eval_recall': 0.88, 'eval_f1': 0.8599348534201954, 'eval_runtime': 12.9973, 'eval_samples_per_second': 29.16, 'eval_steps_per_second': 1.847, 'epoch': 3.0}


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [33]:
import pandas as pd
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch

# 모델 이름 및 토크나이저
model_name = "monologg/koelectra-base-v3-discriminator"
tokenizer_name = "monologg/koelectra-base-v3-discriminator"

# 토크나이저와 모델 로드
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_name)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 감성 분석 함수 정의
def predict_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    return predictions.item()

# 리뷰에 대한 감성 분석 수행 및 레이블링
def label_sentiment(review):
    prediction = predict_sentiment(review)
    # 예측 결과에 따라 긍정(1) 또는 부정(0) 레이블 부여
    if prediction == 1:
        return 1  # 긍정
    else:
        return 0  # 부정

# 각 리뷰에 대해 레이블링을 수행하여 새로운 열에 추가
df1['label'] = df1['review'].apply(label_sentiment)

# 결과 출력
print(df1)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                                                review category restaurant  \
0                                         맛 전혀 특별하지 않음        맛      무명칼국수   
1                     해물칼국수 정말 괜찮음 닭칼은 그닥 만두는 부추맛이 강해서        맛      무명칼국수   
2    Strong point 1 쫄깃한 식감의 수타면 2 아삭하고 감칠맛 나는 겉절이 반...        맛      무명칼국수   
3                                    찐맛집 닭칼이랑 해물칼국수 둘다        맛      무명칼국수   
4    면이랑 국물 미쳤어요 김치도 대존맛 만두도 진짜임 광운대 가면 무조건 가야되는 맛집...        맛      무명칼국수   
..                                                 ...      ...        ...   
170    뭔가 저 말투랑 매너가 기분이 안좋았음 사장과 별개로 파스타는 진짜 개존맛이라 별3개      서비스    라체나1907   
171                             직원분이 친절하시고 음식의 양도 적당하고      서비스    라체나1907   
172                                          서비스가 별로에요      서비스    라체나1907   
173                                           서비스가 좋아요      서비스    라체나1907   
174                                   사장님 너무 불친절해서 싫어요      서비스    라체나1907   

     label  
0        0  
1        0  
2        0  
3        0 

In [34]:
# df1의 라벨에서 0과 1의 갯수 파악
label_counts = df1['label'].value_counts()
print(label_counts)

label
0    173
1      2
Name: count, dtype: int64


In [35]:
import pandas as pd
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch

# labeled 데이터를 로드합니다.
df10 = pd.read_csv('naver_final.csv')

# 모델 이름 및 토크나이저
model_name = "monologg/koelectra-base-v3-discriminator"

# 토크나이저와 모델 로드
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 감성 분석 함수 정의
def predict_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)  # softmax 적용
    positive_probability = probabilities[:, 1].item()  # 긍정 클래스의 확률
    return positive_probability

# 리뷰에 대한 감성 분석 수행 및 레이블링
def label_sentiment(review):
    positive_probability = predict_sentiment(review)
    # 긍정 확률이 0.5 이상이면 긍정(1), 아니면 부정(0)으로 레이블 부여
    if positive_probability >= 0.5:
        return 1  # 긍정
    else:
        return 0  # 부정

# 각 리뷰에 대해 레이블링을 수행하여 새로운 열에 추가
df10['label'] = df10['review'].apply(label_sentiment)

# 결과 출력
print(df10.head())


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                    review category restaurant  label
0  마제소바 준비하신다고 테스트로 맛보여주셔서        맛        윤스쿡      1
1             양도 많고 맛도 있어요        맛        윤스쿡      1
2                  양많고 맛나요        맛        윤스쿡      1
3         학교앞 돈가스중 제일 맛있움ㅜ        맛        윤스쿡      1
4                  너무 맛나네요        맛        윤스쿡      1


In [36]:
# df10의 라벨에서 0과 1의 갯수 파악
label_counts = df10['label'].value_counts()
print(label_counts)

label
1    690
0      7
Name: count, dtype: int64


In [18]:
# 데이터프레임 로드
df3 = pd.read_csv('sentiment_score_data.csv')

df3

FileNotFoundError: [Errno 2] No such file or directory: 'sentiment_score_data.csv'

In [ ]:
# 감성 분석을 위한 파이프라인을 만듭니다.
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# 데이터를 로드합니다.
df3 = pd.read_csv('sentiment_score_data.csv')

# 각 음식점 별로 플랫폼별 리뷰 수를 계산합니다.
restaurant_counts = df3.groupby(['restaurant', 'platform']).size().unstack(fill_value=0)

# 가중치를 계산합니다.
restaurant_counts['total'] = restaurant_counts.sum(axis=1)
restaurant_counts['naver_weight'] = restaurant_counts['naver'] / restaurant_counts['total']
restaurant_counts['kakao_weight'] = restaurant_counts['kakao'] / restaurant_counts['total']

# 별점을 재계산합니다.
def calculate_weighted_score(row):
    restaurant = row['restaurant']
    naver_weight = restaurant_counts.at[restaurant, 'naver_weight']
    kakao_weight = restaurant_counts.at[restaurant, 'kakao_weight']
    if row['platform'] == 'naver':
        weight = naver_weight
    else:
        weight = kakao_weight
    return row['sentiment_score'] * weight

df3['weighted_score'] = df3.apply(calculate_weighted_score, axis=1)

# 음식점 별로 가중치를 적용한 별점을 계산합니다.
restaurant_scores = df3.groupby('restaurant')['weighted_score'].sum().reset_index()

print(restaurant_scores)
